In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import os
import copy
import hist
from coffea import util
import numpy as np
import itertools
import pandas as pd
import mplhep as hep
import HelpfulPlottingDefs as hpd

In [ ]:
Year = 2016
btagWP = 'MediumBTag'
btagWPold = ''
vfp = 'postVFP'
saveOn = True

In [ ]:
dir1 = 'CoffeaOutputsForCombine/Coffea_FirstRun/'

if btagWP != '':
    btagdir = f'{btagWP}/'
else:
    btagdir = btagWP
    
yeardir = f'{str(Year)}/'
btagType = 'DeepCSV'
APVDir = {
    'preVFP': 'APV/',
    'postVFP': 'noAPV/'
}

od = ''
oddir = ''
if btagdir == '':
    od = '_oldANdisc'
    oddir = f'Old{str(Year)}_MediumBTag/'

In [ ]:
import LoadData as LD

JetHT_unweighted = LD.JetHT_Unweighted(btagWP, Year)
JetHT_unweighted_old = LD.JetHT_Unweighted(btagWPold, Year)

In [ ]:
import LoadMC as LM

TTbar_unweighted = LM.TTbar_Unweighted(btagWP, Year)
TTbar_unweighted_old = LM.TTbar_Unweighted(btagWPold, Year)

# Old Taggers Data Cutflow

In [ ]:
LD.Cutflow(JetHT_unweighted_old)

# New Taggers Data Cutflow

In [ ]:
LD.Cutflow(JetHT_unweighted)

# Old Taggers TTbar Cutflow

In [ ]:
LM.Cutflow(TTbar_unweighted_old)

# New Taggers TTbar Cutflow

In [ ]:
LM.Cutflow(TTbar_unweighted)

In [ ]:
import matplotlib.pyplot as plt
import warnings
import re # regular expressions
warnings.filterwarnings("ignore")

In [ ]:
# ---- Optional to rescale x-axis of mistag rates ---- #
def forward(x):
    return x**(1/8)

def inverse(x):
    return x**8

In [ ]:
Lum2016 = 35920. # pb^-1 from https://twiki.cern.ch/twiki/bin/viewauth/CMS/PdmVAnalysisSummaryTable
Lum2017 = 41530.
Lum2018 = 59740.
Lum     = 137190.

t_BR = 0.6741
ttbar_BR = 0.4544 #PDG 2019
ttbar_xs1 = 831.76 * (0.09210) #pb For ttbar mass from 700 to 1000
ttbar_xs2 = 831.76 * (0.02474) #pb For ttbar mass from 1000 to Inf
toptag_sf = 0.9
toptag_kf = 0.7

if Year == 2016:
    if vfp == 'preVFP':
        Lum_frac = 19800. 
    else:
        Lum_frac = Lum2016 - 19800.
        
elif Year == 2017:
    Lum_frac = Lum2017
    
elif Year == 2018:
    Lum_frac = Lum2018
    
else:
    Lum_frac = Lum

# =========== SF =========== #
ttbar_sf1 = Lum_frac*ttbar_xs1*toptag_sf**2*toptag_kf/TTbar_unweighted[f'700_1000_{vfp}']['cutflow']['sumw']
ttbar_sf2 = Lum_frac*ttbar_xs2*toptag_sf**2*toptag_kf/TTbar_unweighted[f'1000_Inf_{vfp}']['cutflow']['sumw']

ttbar_sf1_old = Lum_frac*ttbar_xs1*toptag_sf**2*toptag_kf/TTbar_unweighted_old[f'700_1000_{vfp}']['cutflow']['sumw']
ttbar_sf2_old = Lum_frac*ttbar_xs2*toptag_sf**2*toptag_kf/TTbar_unweighted_old[f'1000_Inf_{vfp}']['cutflow']['sumw']

ttbar_scales = [ttbar_sf1, ttbar_sf2]
ttbar_scales_old = [ttbar_sf1_old, ttbar_sf2_old]

In [ ]:
maindirectory = os.getcwd() 
SaveDirectory = f'{maindirectory}/UnweightedAnalysisPlots/MistagComparisons/{yeardir}{btagdir}{oddir}{APVDir[vfp]}' # split histograms into subdirectories
hpd.DoesDirectoryExist(SaveDirectory) # no need to create the directory several times if it exists already

In [ ]:
# =================================================================================================== #

fig, (ax, bx) = plt.subplots(
    nrows = 1,
    ncols = 2,
    figsize=(20,5),
    sharex=True,
    sharey=True
)
HIP = ''
if vfp == 'preVFP':
    HIP = ' (with HIP)'
title = f'{str(Year)}{HIP} Mistag Rate'

# =================================================================================================== #

Inc0b = hpd.GetMistagInfo(JetHT_unweighted, 0, Year, vfp)
Inc1b = hpd.GetMistagInfo(JetHT_unweighted, 1, Year, vfp)
Inc2b = hpd.GetMistagInfo(JetHT_unweighted, 2, Year, vfp)

Inc0b_old = hpd.GetMistagInfo(JetHT_unweighted_old, 0, Year, vfp)
Inc1b_old = hpd.GetMistagInfo(JetHT_unweighted_old, 1, Year, vfp)
Inc2b_old = hpd.GetMistagInfo(JetHT_unweighted_old, 2, Year, vfp)
            
# =================================================================================================== #

mistag_inclusive0b = hpd.plotratio(Inc0b['Num'], Inc0b['Denom'], ax = ax, histtype='errorbar', 
                           marker='.', markersize=13., color='r')

mistag_inclusive1b = hpd.plotratio(Inc1b['Num'], Inc1b['Denom'], ax = ax, histtype='errorbar', 
                           marker='s', markersize=7., color='g')

mistag_inclusive2b = hpd.plotratio(Inc2b['Num'], Inc2b['Denom'], ax = ax, histtype='errorbar', 
                           marker='^', markersize=7., color='b')

mistag_inclusive0b_old = hpd.plotratio(Inc0b_old['Num'], Inc0b_old['Denom'], ax = bx, histtype='errorbar', 
                           marker='.', markersize=13., color='r')

mistag_inclusive1b_old = hpd.plotratio(Inc1b_old['Num'], Inc1b_old['Denom'], ax = bx, histtype='errorbar', 
                           marker='s', markersize=7., color='g')

mistag_inclusive2b_old = hpd.plotratio(Inc2b_old['Num'], Inc2b_old['Denom'], ax = bx, histtype='errorbar', 
                           marker='^', markersize=7., color='b')

# =================================================================================================== #

plt.ylim(bottom = 0.0, top = 0.35)
plt.xlim([400,7000])

ax.set_ylabel('t-tag Mistag Rates')
ax.set_title(title)
bx.set_title(title + ' Old Taggers')
leg = bx.legend(labels=["0b", "1b", "2b"],fontsize='x-large')

# ---- Optional x-axis scaling ---- #
ax.set_xscale('function', functions=(forward, inverse))
plt.xticks(np.array([400, 500, 600, 700, 800, 900, 1000, 2000, 3000, 4000, 5000]))
ax.set_xticklabels(['', 500, '', '', '', '', 1000, 2000, 3000, 4000, 5000])

#---- Labeling ----#
Lint = str(Lum_frac*.001) # Integrated Luminosity
lumi = plt.text(1.0, 1.08, "L = " + Lint[:6] + " fb$^{-1}$",
        fontsize='xx-large',
        horizontalalignment='right',
        verticalalignment='top',
        transform=ax.transAxes
       )
lumi2 = plt.text(1.0, 1.08, "L = " + Lint[:6] + " fb$^{-1}$",
        fontsize='xx-large',
        horizontalalignment='right',
        verticalalignment='top',
        transform=bx.transAxes
       )
CMS = plt.text(0.01, 0.98, 'CMS Preliminary',
        fontsize='xx-large',
        horizontalalignment='left',
        verticalalignment='top',
        transform=ax.transAxes
       )
CMS2 = plt.text(0.01, 0.98, 'CMS Preliminary',
        fontsize='xx-large',
        horizontalalignment='left',
        verticalalignment='top',
        transform=bx.transAxes
       )
rap = plt.text(0.05, 0.86, 'y-inclusive',
        fontsize='large',
        horizontalalignment='left',
        verticalalignment='top',
        transform=ax.transAxes
       )
rap2 = plt.text(0.05, 0.86, 'y-inclusive',
        fontsize='large',
        horizontalalignment='left',
        verticalalignment='top',
        transform=bx.transAxes
       )
coffee = plt.text(1.0, 0.87, u"☕",
              fontsize=50,
              horizontalalignment='left',
              verticalalignment='bottom',
              transform=bx.transAxes
             )

# =================================================================================================== #
if saveOn:
    filename = f'InclusiveMistag.png'
    plt.savefig(SaveDirectory+filename, bbox_inches="tight")
    print('\n' + SaveDirectory + filename + ' saved')

In [ ]:
# =================================================================================================== #

fig, (ax, bx) = plt.subplots(
    nrows = 1,
    ncols = 2,
    figsize=(20,5),
    sharex=True,
    sharey=True
)
HIP = ''
if vfp == 'preVFP':
    HIP = ' (with HIP)'
title = f'{str(Year)}{HIP} Mistag Rate'

# =================================================================================================== #

inc0b = hpd.GetMistagInfoCR(TTbar_unweighted, JetHT_unweighted, 0, Year, vfp, ttbar_scales)
inc1b = hpd.GetMistagInfoCR(TTbar_unweighted, JetHT_unweighted, 1, Year, vfp, ttbar_scales)
inc2b = hpd.GetMistagInfoCR(TTbar_unweighted, JetHT_unweighted, 2, Year, vfp, ttbar_scales)

inc0b_old = hpd.GetMistagInfoCR(TTbar_unweighted_old, JetHT_unweighted_old, 0, Year, vfp, ttbar_scales_old)
inc1b_old = hpd.GetMistagInfoCR(TTbar_unweighted_old, JetHT_unweighted_old, 1, Year, vfp, ttbar_scales_old)
inc2b_old = hpd.GetMistagInfoCR(TTbar_unweighted_old, JetHT_unweighted_old, 2, Year, vfp, ttbar_scales_old)
            
# =================================================================================================== #

mistag_inclusive0b_cr = hpd.plotratio(inc0b['Num'], inc0b['Denom'], ax = ax, histtype='errorbar', 
                           marker='.', markersize=13., color='r')

mistag_inclusive1b_cr = hpd.plotratio(inc1b['Num'], inc1b['Denom'], ax = ax, histtype='errorbar', 
                           marker='s', markersize=7., color='g')

mistag_inclusive2b_cr = hpd.plotratio(inc2b['Num'], inc2b['Denom'], ax = ax, histtype='errorbar', 
                           marker='^', markersize=7., color='b')

mistag_inclusive0b_cr_old = hpd.plotratio(inc0b_old['Num'], inc0b_old['Denom'], ax = bx, histtype='errorbar', 
                           marker='.', markersize=13., color='r')

mistag_inclusive1b_cr_old = hpd.plotratio(inc1b_old['Num'], inc1b_old['Denom'], ax = bx, histtype='errorbar', 
                           marker='s', markersize=7., color='g')

mistag_inclusive2b_cr_old = hpd.plotratio(inc2b_old['Num'], inc2b_old['Denom'], ax = bx, histtype='errorbar', 
                           marker='^', markersize=7., color='b')

# =================================================================================================== #

plt.ylim(bottom = 0.0, top = 0.35)
plt.xlim([400,7000])

ax.set_ylabel('t-tag Mistag Rates')
ax.set_title(title)
bx.set_title(title + ' Old Taggers')
leg = bx.legend(labels=["0b", "1b", "2b"],fontsize='x-large')

# ---- Optional x-axis scaling ---- #
ax.set_xscale('function', functions=(forward, inverse))
plt.xticks(np.array([400, 500, 600, 700, 800, 900, 1000, 2000, 3000, 4000, 5000]))
ax.set_xticklabels(['', 500, '', '', '', '', 1000, 2000, 3000, 4000, 5000])

#---- Labeling ----#
Lint = str(Lum_frac*.001) # Integrated Luminosity
lumi = plt.text(1.0, 1.08, "L = " + Lint[:6] + " fb$^{-1}$",
        fontsize='xx-large',
        horizontalalignment='right',
        verticalalignment='top',
        transform=ax.transAxes
       )
lumi2 = plt.text(1.0, 1.08, "L = " + Lint[:6] + " fb$^{-1}$",
        fontsize='xx-large',
        horizontalalignment='right',
        verticalalignment='top',
        transform=bx.transAxes
       )
CMS = plt.text(0.01, 0.98, 'CMS Preliminary',
        fontsize='xx-large',
        horizontalalignment='left',
        verticalalignment='top',
        transform=ax.transAxes
       )
CMS2 = plt.text(0.01, 0.98, 'CMS Preliminary',
        fontsize='xx-large',
        horizontalalignment='left',
        verticalalignment='top',
        transform=bx.transAxes
       )
rap = plt.text(0.05, 0.86, 'y-inclusive',
        fontsize='large',
        horizontalalignment='left',
        verticalalignment='top',
        transform=ax.transAxes
       )
rap2 = plt.text(0.05, 0.86, 'y-inclusive',
        fontsize='large',
        horizontalalignment='left',
        verticalalignment='top',
        transform=bx.transAxes
       )
contam = plt.text(0.05, 0.81, r'$t\bar{t}$ contam. removed',
        fontsize='large',
        horizontalalignment='left',
        verticalalignment='top',
        transform=ax.transAxes
       )
contam2 = plt.text(0.05, 0.81, r'$t\bar{t}$ contam. removed',
        fontsize='large',
        horizontalalignment='left',
        verticalalignment='top',
        transform=bx.transAxes
       )
coffee = plt.text(1.0, 0.87, u"☕",
              fontsize=50,
              horizontalalignment='left',
              verticalalignment='bottom',
              transform=bx.transAxes
             )

# =================================================================================================== #
if saveOn:
    filename = f'InclusiveMistag_ContamRemoved.png'
    plt.savefig(SaveDirectory+filename, bbox_inches="tight")
    print('\n' + SaveDirectory + filename + ' saved')

In [ ]:
# =================================================================================================== #

fig, ([axDataNum, axDataNumOld, axTTNum, axTTNumOld], [axDataDenom, axDataDenomOld, axTTDenom, axTTDenomOld]) = plt.subplots(
    nrows = 2,
    ncols = 4,
    figsize=(25,10),
    sharex=True
    # sharey=True
)
HIP = ''
if vfp == 'preVFP':
    HIP = ' (with HIP)'
title = f'{str(Year)}{HIP}'

LS = 'dotted'
LW = 2.5

# =================================================================================================== #

Inc0b = hpd.GetMistagInfo(JetHT_unweighted, 0, Year, vfp)
Inc1b = hpd.GetMistagInfo(JetHT_unweighted, 1, Year, vfp)
Inc2b = hpd.GetMistagInfo(JetHT_unweighted, 2, Year, vfp)

Inc0b_old = hpd.GetMistagInfo(JetHT_unweighted_old, 0, Year, vfp)
Inc1b_old = hpd.GetMistagInfo(JetHT_unweighted_old, 1, Year, vfp)
Inc2b_old = hpd.GetMistagInfo(JetHT_unweighted_old, 2, Year, vfp)

Inc0bTT = hpd.GetMistagInfoTTbar(TTbar_unweighted, 0, Year, vfp, ttbar_scales)
Inc1bTT = hpd.GetMistagInfoTTbar(TTbar_unweighted, 1, Year, vfp, ttbar_scales)
Inc2bTT = hpd.GetMistagInfoTTbar(TTbar_unweighted, 2, Year, vfp, ttbar_scales)

Inc0bTT_old = hpd.GetMistagInfoTTbar(TTbar_unweighted_old, 0, Year, vfp, ttbar_scales_old)
Inc1bTT_old = hpd.GetMistagInfoTTbar(TTbar_unweighted_old, 1, Year, vfp, ttbar_scales_old)
Inc2bTT_old = hpd.GetMistagInfoTTbar(TTbar_unweighted_old, 2, Year, vfp, ttbar_scales_old)
            
# =================================================================================================== # 

Inc0b_Num = Inc0b['Num'].plot1d(ax=axDataNum, histtype='step', color='r')
Inc0b['Denom'].plot1d(ax=axDataDenom, histtype='step', color='r')

Inc1b['Num'].plot1d(ax=axDataNum, histtype='step', color='g')
Inc1b['Denom'].plot1d(ax=axDataDenom, histtype='step', color='g')

Inc2b['Num'].plot1d(ax=axDataNum, histtype='step', color='b')
Inc2b['Denom'].plot1d(ax=axDataDenom, histtype='step', color='b')

Inc0b_old['Num'].plot1d(ax=axDataNumOld, histtype='step', color='r')
Inc0b_old['Denom'].plot1d(ax=axDataDenomOld, histtype='step', color='r')

Inc1b_old['Num'].plot1d(ax=axDataNumOld, histtype='step', color='g')
Inc1b_old['Denom'].plot1d(ax=axDataDenomOld, histtype='step', color='g')

Inc2b_old['Num'].plot1d(ax=axDataNumOld, histtype='step', color='b')
Inc2b_old['Denom'].plot1d(ax=axDataDenomOld, histtype='step', color='b')

# =================================================================================================== #

Inc0bTT['Num'].plot1d(ax=axTTNum, histtype='step', linestyle=LS, lw=LW, color='r')
Inc0bTT['Denom'].plot1d(ax=axTTDenom, histtype='step', linestyle=LS, lw=LW, color='r')

Inc1bTT['Num'].plot1d(ax=axTTNum, histtype='step', linestyle=LS, lw=LW, color='g')
Inc1bTT['Denom'].plot1d(ax=axTTDenom, histtype='step', linestyle=LS, lw=LW, color='g')

Inc2bTT['Num'].plot1d(ax=axTTNum, histtype='step', linestyle=LS, lw=LW, color='b')
Inc2bTT['Denom'].plot1d(ax=axTTDenom, histtype='step', linestyle=LS, lw=LW, color='b')

Inc0bTT_old['Num'].plot1d(ax=axTTNumOld, histtype='step', linestyle=LS, lw=LW, color='r')
Inc0bTT_old['Denom'].plot1d(ax=axTTDenomOld, histtype='step', linestyle=LS, lw=LW, color='r')

Inc1bTT_old['Num'].plot1d(ax=axTTNumOld, histtype='step', linestyle=LS, lw=LW, color='g')
Inc1bTT_old['Denom'].plot1d(ax=axTTDenomOld, histtype='step', linestyle=LS, lw=LW, color='g')

Inc2bTT_old['Num'].plot1d(ax=axTTNumOld, histtype='step', linestyle=LS, lw=LW, color='b')
Inc2bTT_old['Denom'].plot1d(ax=axTTDenomOld, histtype='step', linestyle=LS, lw=LW, color='b')

# =================================================================================================== #

# plt.ylim(bottom = 0.0, top = 0.35)
plt.xlim([400,7000])

axDataNum.set_ylabel('Counts')
axDataDenom.set_ylabel('Counts')
axDataNum.set_title(title + ' Data Numerators')
axDataDenom.set_title(title + ' Data Denominators')

axDataNumOld.set_title(title + ' Data Numerators (Old Taggers)')
axDataDenomOld.set_title(title + ' Data Denominators (Old Taggers)')

axTTNum.set_ylabel('Counts')
axTTDenom.set_ylabel('Counts')
axTTNum.set_title(title + ' TTbar Numerators')
axTTDenom.set_title(title + ' TTbar Denominators')

axTTNumOld.set_title(title + ' TTbar Numerators (Old Taggers)')
axTTDenomOld.set_title(title + ' TTbar Denominators (Old Taggers)')

axTTNum.set_ylim(0., -1400.)
axTTDenom.set_ylim(0., -2000.)
axTTNumOld.set_ylim(0., -500.)
axTTDenomOld.set_ylim(0., -1500.)

leg1 = axDataNumOld.legend(labels=["Numerator 0b", "Numerator 1b", "Numerator 2b"],fontsize='x-large')
leg2 = axTTNumOld.legend(labels=["Numerator 0b", "Numerator 1b", "Numerator 2b"],fontsize='x-large')
leg3 = axDataDenomOld.legend(labels=["Denominator 0b", "Denominator 1b", "Denominator 2b"],fontsize='x-large')
leg4 = axTTDenomOld.legend(labels=["Denominator 0b", "Denominator 1b", "Denominator 2b"],fontsize='x-large')

# ---- Optional x-axis scaling ---- #
axDataNum.set_xscale('function', functions=(forward, inverse))
axDataDenomOld.set_xscale('function', functions=(forward, inverse))
plt.xticks(np.array([400, 500, 600, 700, 800, 900, 1000, 2000, 3000, 4000, 5000]))
axDataNum.set_xticklabels(['', 500, '', '', '', '', 1000, 2000, 3000, 4000, 5000])
axDataDenomOld.set_xticklabels(['', 500, '', '', '', '', 1000, 2000, 3000, 4000, 5000])

# =================================================================================================== #
if saveOn:
    filename = f'AllNumeratorsDenominators.png'
    plt.savefig(SaveDirectory+filename, bbox_inches="tight")
    print('\n' + SaveDirectory + filename + ' saved')

In [ ]:
NumData0bOld = np.sum(Inc0b_old['Num'].view().value)
NumData0bNew = np.sum(Inc0b['Num'].view().value)
NumData1bOld = np.sum(Inc1b_old['Num'].view().value)
NumData1bNew = np.sum(Inc1b['Num'].view().value)
NumData2bOld = np.sum(Inc2b_old['Num'].view().value)
NumData2bNew = np.sum(Inc2b['Num'].view().value)


NumTT0bOld = np.sum(Inc0bTT_old['Num'].view().value)
NumTT0bNew = np.sum(Inc0bTT['Num'].view().value)
NumTT1bOld = np.sum(Inc1bTT_old['Num'].view().value)
NumTT1bNew = np.sum(Inc1bTT['Num'].view().value)
NumTT2bOld = np.sum(Inc2bTT_old['Num'].view().value)
NumTT2bNew = np.sum(Inc2bTT['Num'].view().value)

In [ ]:
DenomData0bOld = np.sum(Inc0b_old['Denom'].view().value)
DenomData0bNew = np.sum(Inc0b['Denom'].view().value)
DenomData1bOld = np.sum(Inc1b_old['Denom'].view().value)
DenomData1bNew = np.sum(Inc1b['Denom'].view().value)
DenomData2bOld = np.sum(Inc2b_old['Denom'].view().value)
DenomData2bNew = np.sum(Inc2b['Denom'].view().value)


DenomTT0bOld = np.sum(Inc0bTT_old['Denom'].view().value)
DenomTT0bNew = np.sum(Inc0bTT['Denom'].view().value)
DenomTT1bOld = np.sum(Inc1bTT_old['Denom'].view().value)
DenomTT1bNew = np.sum(Inc1bTT['Denom'].view().value)
DenomTT2bOld = np.sum(Inc2bTT_old['Denom'].view().value)
DenomTT2bNew = np.sum(Inc2bTT['Denom'].view().value)

In [ ]:
PlotType = 'linear'

fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(18,8))

Columns = ['# of b-tags', 'Old Taggers', 'New Taggers', 'Difference']

dfNumData = pd.DataFrame([['0', NumData0bOld, NumData0bNew, NumData0bNew-NumData0bOld], 
                          ['1', NumData1bOld, NumData1bNew, NumData1bNew-NumData1bOld], 
                          ['2', NumData2bOld, NumData2bNew, NumData2bNew-NumData2bOld]],
                  columns=Columns)
dfNumTT = pd.DataFrame([['0', NumTT0bOld, NumTT0bNew, NumTT0bNew-NumTT0bOld], 
                        ['1', NumTT1bOld, NumTT1bNew, NumTT1bNew-NumTT1bOld], 
                        ['2', NumTT2bOld, NumTT2bNew, NumTT2bNew-NumTT2bOld]],
                  columns=Columns)
dfDenomData = pd.DataFrame([['0', DenomData0bOld, DenomData0bNew, DenomData0bNew-DenomData0bOld], 
                            ['1', DenomData1bOld, DenomData1bNew, DenomData1bNew-DenomData1bOld], 
                            ['2', DenomData2bOld, DenomData2bNew, DenomData2bNew-DenomData2bOld]],
                  columns=Columns)
dfDenomTT = pd.DataFrame([['0', DenomTT0bOld, DenomTT0bNew, DenomTT0bNew-DenomTT0bOld], 
                          ['1', DenomTT1bOld, DenomTT1bNew, DenomTT1bNew-DenomTT1bOld], 
                          ['2', DenomTT2bOld, DenomTT2bNew, DenomTT2bNew-DenomTT2bOld]],
                  columns=Columns)

if PlotType == 'symlog':
    axes[0,1].set_ylim(bottom=0., top=-10000.)
    axes[1,1].set_ylim(bottom=0., top=-10000.)
else:
    axes[0,1].set_ylim(bottom=0., top=-5000.)
    axes[1,1].set_ylim(bottom=0., top=-7000.)

axes[0,0].set_yscale(PlotType)
axes[0,1].set_yscale(PlotType)
axes[1,0].set_yscale(PlotType)
axes[1,1].set_yscale(PlotType)

colors = ['green', 'blue', 'red']

dfNumData.plot(#x='# of b-tags',
        kind='bar',
        stacked=False,
        title='Data Numerator Counts from New and Old Taggers',
        color=colors,
        ax=axes[0,0])
dfNumTT.plot(#x='# of b-tags',
        kind='bar',
        stacked=False,
        title='Scaled TTbar Numerator Counts from New and Old Taggers',
        color=colors,
        ax=axes[0,1])
dfDenomData.plot(x='# of b-tags',
        kind='bar',
        stacked=False,
        title='Data Denominator Counts from New and Old Taggers',
        color=colors,
        ax=axes[1,0])
dfDenomTT.plot(x='# of b-tags',
        kind='bar',
        stacked=False,
        title='Scaled TTbar Denominator Counts from New and Old Taggers',
        color=colors,
        ax=axes[1,1])
if saveOn:
    filename = f'Allbtags_NumeratorsDenominators_BarGraphs.png'
    plt.savefig(SaveDirectory+filename, bbox_inches="tight")
    print('\n' + SaveDirectory + filename + ' saved')

In [ ]:
ttbar_scales = [-ttbar_sf1, -ttbar_sf2] # Reverse the negative scaling temporarily for plotting in this cell
ttbar_scales_old = [-ttbar_sf1_old, -ttbar_sf2_old] # Reverse the negative scaling temporarily for plotting in this cell
# =================================================================================================== #

PlotType = 'log'

for i in range(3):
    
    fig, ([axNum, axNumOld], [axDenom, axDenomOld]) = plt.subplots(
        nrows = 2,
        ncols = 2,
        figsize=(25,10),
        sharex=True
        # sharey=True
    )
    HIP = ''
    if vfp == 'preVFP':
        HIP = ' (with HIP)'
    title = f'{str(Year)}{HIP}'
    subtitle = f'{i}-btag region'
    
    plt.suptitle(subtitle, fontsize='xx-large')

    LS = 'dotted'
    LW = 2.5

    # =================================================================================================== #

    Inc0b = hpd.GetMistagInfo(JetHT_unweighted, 0, Year, vfp)
    Inc1b = hpd.GetMistagInfo(JetHT_unweighted, 1, Year, vfp)
    Inc2b = hpd.GetMistagInfo(JetHT_unweighted, 2, Year, vfp)

    Inc0b_old = hpd.GetMistagInfo(JetHT_unweighted_old, 0, Year, vfp)
    Inc1b_old = hpd.GetMistagInfo(JetHT_unweighted_old, 1, Year, vfp)
    Inc2b_old = hpd.GetMistagInfo(JetHT_unweighted_old, 2, Year, vfp)

    Inc0bTT = hpd.GetMistagInfoTTbar(TTbar_unweighted, 0, Year, vfp, ttbar_scales)
    Inc1bTT = hpd.GetMistagInfoTTbar(TTbar_unweighted, 1, Year, vfp, ttbar_scales)
    Inc2bTT = hpd.GetMistagInfoTTbar(TTbar_unweighted, 2, Year, vfp, ttbar_scales)

    Inc0bTT_old = hpd.GetMistagInfoTTbar(TTbar_unweighted_old, 0, Year, vfp, ttbar_scales_old)
    Inc1bTT_old = hpd.GetMistagInfoTTbar(TTbar_unweighted_old, 1, Year, vfp, ttbar_scales_old)
    Inc2bTT_old = hpd.GetMistagInfoTTbar(TTbar_unweighted_old, 2, Year, vfp, ttbar_scales_old)

    # =================================================================================================== # 
    
    if i == 0:
        Inc0b_Num = Inc0b['Num'].plot1d(ax=axNum, histtype='step', color='r')
        Inc0b['Denom'].plot1d(ax=axDenom, histtype='step', color='r')
        Inc0b_old['Num'].plot1d(ax=axNumOld, histtype='step', color='r')
        Inc0b_old['Denom'].plot1d(ax=axDenomOld, histtype='step', color='r')
        axNum.set_ylim(bottom=1., top=10**7)
        axDenom.set_ylim(bottom=1., top=10**7)
        axNumOld.set_ylim(bottom=1., top=10**7)
        axDenomOld.set_ylim(bottom=1., top=10**7)

    elif i == 1:
        Inc1b['Num'].plot1d(ax=axNum, histtype='step', color='g')
        Inc1b['Denom'].plot1d(ax=axDenom, histtype='step', color='g')
        Inc1b_old['Num'].plot1d(ax=axNumOld, histtype='step', color='g')
        Inc1b_old['Denom'].plot1d(ax=axDenomOld, histtype='step', color='g')
        axNum.set_ylim(bottom=1., top=10**6)
        axDenom.set_ylim(bottom=1., top=10**6)
        axNumOld.set_ylim(bottom=1., top=10**6)
        axDenomOld.set_ylim(bottom=1., top=10**6)

    else:
        Inc2b['Num'].plot1d(ax=axNum, histtype='step', color='b')
        Inc2b['Denom'].plot1d(ax=axDenom, histtype='step', color='b')
        Inc2b_old['Num'].plot1d(ax=axNumOld, histtype='step', color='b')
        Inc2b_old['Denom'].plot1d(ax=axDenomOld, histtype='step', color='b')
        axNum.set_ylim(bottom=1., top=10**5)
        axDenom.set_ylim(bottom=1., top=10**5)
        axNumOld.set_ylim(bottom=1., top=10**5)
        axDenomOld.set_ylim(bottom=1., top=10**5)

    # =================================================================================================== #
    if i == 0:
        Inc0bTT['Num'].plot1d(ax=axNum, histtype='step', linestyle=LS, lw=LW, color='r')
        Inc0bTT['Denom'].plot1d(ax=axDenom, histtype='step', linestyle=LS, lw=LW, color='r')
        Inc0bTT_old['Num'].plot1d(ax=axNumOld, histtype='step', linestyle=LS, lw=LW, color='r')
        Inc0bTT_old['Denom'].plot1d(ax=axDenomOld, histtype='step', linestyle=LS, lw=LW, color='r')

    elif i == 1:
        Inc1bTT['Num'].plot1d(ax=axNum, histtype='step', linestyle=LS, lw=LW, color='g')
        Inc1bTT['Denom'].plot1d(ax=axDenom, histtype='step', linestyle=LS, lw=LW, color='g')
        Inc1bTT_old['Num'].plot1d(ax=axNumOld, histtype='step', linestyle=LS, lw=LW, color='g')
        Inc1bTT_old['Denom'].plot1d(ax=axDenomOld, histtype='step', linestyle=LS, lw=LW, color='g')

    else:
        Inc2bTT['Num'].plot1d(ax=axNum, histtype='step', linestyle=LS, lw=LW, color='b')
        Inc2bTT['Denom'].plot1d(ax=axDenom, histtype='step', linestyle=LS, lw=LW, color='b')
        Inc2bTT_old['Num'].plot1d(ax=axNumOld, histtype='step', linestyle=LS, lw=LW, color='b')
        Inc2bTT_old['Denom'].plot1d(ax=axDenomOld, histtype='step', linestyle=LS, lw=LW, color='b')

    # =================================================================================================== #

    # plt.ylim(bottom = 0.0, top = 0.35)
    plt.xlim([400,7000])

    axNum.set_ylabel('Counts')
    axDenom.set_ylabel('Counts')
    axNum.set_title(title + ' Numerators')
    axDenom.set_title(title + ' Denominators')
    axNum.set_yscale(PlotType)
    axDenom.set_yscale(PlotType)

    axNumOld.set_title(title + ' Data Numerators (Old Taggers)')
    axDenomOld.set_title(title + ' Data Denominators (Old Taggers)')
    axNumOld.set_yscale(PlotType)
    axDenomOld.set_yscale(PlotType)

    leg1 = axNum.legend(labels=["Data", "TTbar"],fontsize='x-large')
    leg2 = axDenom.legend(labels=["Data", "TTbar"],fontsize='x-large')
    leg3 = axNumOld.legend(labels=["Data", "TTbar"],fontsize='x-large')
    leg4 = axDenomOld.legend(labels=["Data", "TTbar"],fontsize='x-large')

    # ---- Optional x-axis scaling ---- #
    axNum.set_xscale('function', functions=(forward, inverse))
    axDenomOld.set_xscale('function', functions=(forward, inverse))
    plt.xticks(np.array([400, 500, 600, 700, 800, 900, 1000, 2000, 3000, 4000, 5000]))
    axNum.set_xticklabels(['', 500, '', '', '', '', 1000, 2000, 3000, 4000, 5000])
    axDenomOld.set_xticklabels(['', 500, '', '', '', '', 1000, 2000, 3000, 4000, 5000])

    # =================================================================================================== #
    
    if saveOn:
        filename = f'{i}btag_NumeratorsDenominators_BarGraphs.png'
        plt.savefig(SaveDirectory+filename, bbox_inches="tight")
        print('\n' + SaveDirectory + filename + ' saved')
    
ttbar_scales = [ttbar_sf1, ttbar_sf2] # Put scales back to normal
ttbar_scales_old = [ttbar_sf1_old, ttbar_sf2_old] # Put scales back to normal

In [ ]:
NumTT0bOld = np.sum(Inc0bTT_old['Num'].view().value)
NumTT0bNew = np.sum(Inc0bTT['Num'].view().value)
NumTT1bOld = np.sum(Inc1bTT_old['Num'].view().value)
NumTT1bNew = np.sum(Inc1bTT['Num'].view().value)
NumTT2bOld = np.sum(Inc2bTT_old['Num'].view().value)
NumTT2bNew = np.sum(Inc2bTT['Num'].view().value)

In [ ]:
DenomTT0bOld = np.sum(Inc0bTT_old['Denom'].view().value)
DenomTT0bNew = np.sum(Inc0bTT['Denom'].view().value)
DenomTT1bOld = np.sum(Inc1bTT_old['Denom'].view().value)
DenomTT1bNew = np.sum(Inc1bTT['Denom'].view().value)
DenomTT2bOld = np.sum(Inc2bTT_old['Denom'].view().value)
DenomTT2bNew = np.sum(Inc2bTT['Denom'].view().value)

In [ ]:
PlotType = 'log'

fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(18,8))

Columns = ['# of b-tags', 'Data', 'TTbar', 'Difference']

dfNumNew = pd.DataFrame([['0', NumData0bNew, NumTT0bNew, NumData0bNew-NumTT0bNew], 
                         ['1', NumData1bNew, NumTT1bNew, NumData1bNew-NumTT1bNew], 
                         ['2', NumData2bNew, NumTT2bNew, NumData2bNew-NumTT2bNew]],
                  columns=Columns)
dfNumOld = pd.DataFrame([['0', NumData0bOld, NumTT0bOld, NumData0bOld-NumTT0bOld], 
                         ['1', NumData1bOld, NumTT1bOld, NumData1bOld-NumTT1bOld], 
                         ['2', NumData2bOld, NumTT2bOld, NumData2bOld-NumTT2bOld]],
                  columns=Columns)
dfDenomNew = pd.DataFrame([['0', DenomData0bNew, DenomTT0bNew, DenomData0bNew-DenomTT0bNew], 
                           ['1', DenomData1bNew, DenomTT1bNew, DenomData1bNew-DenomTT1bNew], 
                           ['2', DenomData2bNew, DenomTT2bNew, DenomData2bNew-DenomTT2bNew]],
                  columns=Columns)
dfDenomOld = pd.DataFrame([['0', DenomData0bOld, DenomTT0bOld, DenomData0bOld-DenomTT0bOld], 
                           ['1', DenomData1bOld, DenomTT1bOld, DenomData1bOld-DenomTT1bOld], 
                           ['2', DenomData2bOld, DenomTT2bOld, DenomData2bOld-DenomTT2bOld]],
                  columns=Columns)


axes[0,0].set_ylim(bottom=1., top=10**8)
axes[0,1].set_ylim(bottom=1., top=10**8)
axes[1,0].set_ylim(bottom=1., top=10**8)
axes[1,1].set_ylim(bottom=1., top=10**8)

axes[0,0].set_yscale(PlotType)
axes[0,1].set_yscale(PlotType)
axes[1,0].set_yscale(PlotType)
axes[1,1].set_yscale(PlotType)

colors = ['green', 'blue', 'red']

dfNumNew.plot(#x='# of b-tags',
        kind='bar',
        stacked=False,
        title='Numerator Counts',
        color=colors,
        ax=axes[0,0])
dfNumOld.plot(#x='# of b-tags',
        kind='bar',
        stacked=False,
        title='Numerator Counts from Old Taggers',
        color=colors,
        ax=axes[0,1])
dfDenomNew.plot(x='# of b-tags',
        kind='bar',
        stacked=False,
        title='Denominator Counts',
        color=colors,
        ax=axes[1,0])
dfDenomOld.plot(x='# of b-tags',
        kind='bar',
        stacked=False,
        title='Denominator Counts from Old Taggers',
        color=colors,
        ax=axes[1,1])

if saveOn:
    filename = f'Allbtags_NumeratorsDenominators_BarGraphs2.png'
    plt.savefig(SaveDirectory+filename, bbox_inches="tight")
    print('\n' + SaveDirectory + filename + ' saved')
print((NumData0bNew-NumTT0bNew)/(DenomData0bNew-DenomTT0bNew))